In [5]:
import torch
import torch.nn as nn
from torchvision import models

import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
import numpy as np
from prepare_data import train_loader, val_loader, test_loader


Number of classes: 31
✅ Dataset split completed successfully!
Classes: ['Akshay_Kumar', 'Alexandra_Daddario', 'Alia_Bhatt', 'Amitabh_Bachchan', 'Andy_Samberg', 'Anushka_Sharma', 'Billie_Eilish', 'Brad_Pitt', 'Camila_Cabello', 'Charlize_Theron', 'Claire_Holt', 'Courtney_Cox', 'Dwayne_Johnson', 'Elizabeth_Olsen', 'Ellen_Degeneres', 'Henry_Cavill', 'Hrithik_Roshan', 'Hugh_Jackman', 'Jessica_Alba', 'Kashyap', 'Lisa_Kudrow', 'Margot_Robbie', 'Marmik', 'Natalie_Portman', 'Priyanka_Chopra', 'Robert_Downey_Jr', 'Roger_Federer', 'Tom_Cruise', 'Vijay_Deverakonda', 'Virat_Kohli', 'Zac_Efron']
Number of training images: 2536
Number of validation images: 1224
Number of test images: 1266


In [6]:

device = torch.device("cpu")

# Load pre-trained VGG19
vgg19_tl = models.vgg19(pretrained=True)

# Freeze convolutional layers
for param in vgg19_tl.features.parameters():
    param.requires_grad = False

# Replace classifier for 31 classes
vgg19_tl.classifier = nn.Sequential(
    nn.Linear(512*7*7, 4096),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(4096, 4096),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(4096, 31)
)

vgg19_tl = vgg19_tl.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vgg19_tl.classifier.parameters(), lr=1e-4)
num_epochs = 10

/home/doaa/miniconda3/lib/python3.13/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/doaa/miniconda3/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
for epoch in range(num_epochs):
    vgg19_tl.train()
    train_loss = 0
    all_preds = []
    all_labels = []

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = vgg19_tl(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        all_preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

    train_acc = accuracy_score(all_labels, all_preds)
    train_precision = precision_score(all_labels, all_preds, average='weighted')
    train_recall = recall_score(all_labels, all_preds, average='weighted')
    train_f1 = f1_score(all_labels, all_preds, average='weighted')

    vgg19_tl.eval()
    val_preds = []
    val_labels = []
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = vgg19_tl(images)
            val_preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    val_acc = accuracy_score(val_labels, val_preds)
    val_precision = precision_score(val_labels, val_preds, average='weighted')
    val_recall = recall_score(val_labels, val_preds, average='weighted')
    val_f1 = f1_score(val_labels, val_preds, average='weighted')

    print(f"Epoch [{epoch+1}/{num_epochs}] "
          f"Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f} | "
          f"Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, F1: {train_f1:.4f}")


/home/doaa/miniconda3/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch [1/10] Train Acc: 0.1936, Val Acc: 0.4265 | Precision: 0.1818, Recall: 0.1936, F1: 0.1818


/home/doaa/miniconda3/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch [2/10] Train Acc: 0.3797, Val Acc: 0.5384 | Precision: 0.3671, Recall: 0.3797, F1: 0.3687
Epoch [3/10] Train Acc: 0.4945, Val Acc: 0.6119 | Precision: 0.4891, Recall: 0.4945, F1: 0.4875


/home/doaa/miniconda3/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch [4/10] Train Acc: 0.5655, Val Acc: 0.6283 | Precision: 0.5597, Recall: 0.5655, F1: 0.5610
Epoch [5/10] Train Acc: 0.6151, Val Acc: 0.7083 | Precision: 0.6114, Recall: 0.6151, F1: 0.6109
Epoch [6/10] Train Acc: 0.6443, Val Acc: 0.7206 | Precision: 0.6419, Recall: 0.6443, F1: 0.6421
Epoch [7/10] Train Acc: 0.6814, Val Acc: 0.7443 | Precision: 0.6791, Recall: 0.6814, F1: 0.6789
Epoch [8/10] Train Acc: 0.6972, Val Acc: 0.7598 | Precision: 0.6955, Recall: 0.6972, F1: 0.6957
Epoch [9/10] Train Acc: 0.7271, Val Acc: 0.8015 | Precision: 0.7261, Recall: 0.7271, F1: 0.7258
Epoch [10/10] Train Acc: 0.7551, Val Acc: 0.8448 | Precision: 0.7548, Recall: 0.7551, F1: 0.7547
